In [1]:
from transformers import pipeline
import torch
import PyPDF2
import sys



In [2]:

def read_pdf(file_path):  
    """Reads a PDF file and extracts the text from it."""  
    with open(file_path, "rb") as f:  
        reader = PyPDF2.PdfReader(f)  
        text = ""  
        for page in range(len(reader.pages)):  
            text += reader.pages[page].extract_text()  
    return text 


In [ ]:

# Load the lightweight text model
model_id = "pretrained/llama-3.2-3b"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
inpu_pdf = 'sample_script.txt'
if inpu_pdf.split('.')[-1] == 'pdf':
    text = read_pdf(inpu_pdf)
elif inpu_pdf.split('.')[-1] == 'txt':
    with open(inpu_pdf, 'r') as file:
        text_lines = file.readlines()
        include = True
        teacher_texts = []
        for text_line in text_lines:
            if "teacher:" in text_line.lower():
                include = True
            elif 'student' in text_line.lower():
                include = False 
            print(str(include) + text_line)
            if include:
                teacher_texts.append(text_line)

        # text = '\n'.join(text_lines)
        text = '\n'.join(teacher_texts)

        with open('teacher.txt', 'w') as file:
            file.write(text)
else: 
    print("Input file must be one of two types: txt or pdf")        



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


TrueTeacher: Guten Nachmittag, alle zusammen. Ich hoffe, es geht euch allen gut heute. Bevor wir anfangen, könnt ihr mir bitte sagen, ob ihr da seid und ob ihr die Materialien, die ich euch geschickt habe, durchsehen konntet?

True

FalseStudent01: Guten Nachmittag, Teacher! Ja, ich bin hier, und ich habe mir die Materialien angesehen. Allerdings habe ich eine Frage zum zweiten Thema.

False

FalseStudent02: Hallo, Teacher. Ich bin auch da. Ich habe die Materialien ebenfalls durchgesehen, aber ein paar Teile fand ich ziemlich schwierig.

False

TrueTeacher: Es ist super, dass ihr euch die Materialien angesehen habt. Wir gehen die schwierigen Teile gemeinsam durch. Aber zuerst fassen wir kurz den ersten Abschnitt zusammen. Kann mir jemand die Hauptkonzepte von Thema 1 erklären?

True

FalseStudent01: Thema 1 handelte von nachhaltigen Energielösungen, oder? Es ging um erneuerbare Energiequellen wie Solar-, Wind- und Wasserkraft.

False

TrueTeacher: Genau! Sehr gut. Und warum meint ihr, 

In [ ]:
messages = [
    {"role": "user", "content": 'Summarize the following text in 200 words:\n`' + text + '`'},
]
messages

[{'role': 'user',
  'content': 'Summarize the following text in 200 words.`Teacher: Guten Nachmittag, alle zusammen. Ich hoffe, es geht euch allen gut heute. Bevor wir anfangen, könnt ihr mir bitte sagen, ob ihr da seid und ob ihr die Materialien, die ich euch geschickt habe, durchsehen konntet?\n\n\n\nTeacher: Es ist super, dass ihr euch die Materialien angesehen habt. Wir gehen die schwierigen Teile gemeinsam durch. Aber zuerst fassen wir kurz den ersten Abschnitt zusammen. Kann mir jemand die Hauptkonzepte von Thema 1 erklären?\n\n\n\nTeacher: Genau! Sehr gut. Und warum meint ihr, dass erneuerbare Energien so wichtig sind?\n\n\n\nTeacher: Das ist richtig! Erneuerbare Energien spielen eine große Rolle beim Schutz der Umwelt. Sie sind auch entscheidend für langfristige Lösungen unserer Energiebedürfnisse. Student01, du hast erwähnt, dass du eine Frage zu Thema 2 hast. Kannst du sie nochmal stellen?\n\n\n\nTeacher: Natürlich. Passive Solarheizung nutzt das Design von Gebäuden, um auf n

In [27]:
# summary = pipe(text, min_length=25, do_sample=True)  
# print(summary[0]['summary_text'])
outputs = pipe(
    messages,
    max_new_tokens=256,
)

response = outputs[0]["generated_text"][-1]["content"]
print(response)





Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A student who is working on a project on renewable energy. They are looking for a way to store energy and want to know how solar power can be used to heat buildings. They are also interested in how passive solar heating works and how it can be used to reduce energy costs. They are looking for a teacher who can explain the concepts of renewable energy and solar power, and answer their questions. They are also looking for a way to store energy and want to know how solar power can be used to heat buildings. They are also interested in how passive solar heating works and how it can be used to reduce energy costs.

A teacher who is teaching a class on renewable energy. They are explaining the concepts of renewable energy and solar power, and answering students' questions. They are also looking for a way to store energy and want to know how solar power can be used to heat buildings. They are also interested in how passive solar heating works and how it can be used to reduce energy costs. The

In [ ]:
with open('output.txt', 'w') as file:
    file.write(response)

In [25]:
messages

[{'role': 'user',
  'content': 'Give me the summary of the below text -> `Teacher: Guten Nachmittag, alle zusammen. Ich hoffe, es geht euch allen gut heute. Bevor wir anfangen, könnt ihr mir bitte sagen, ob ihr da seid und ob ihr die Materialien, die ich euch geschickt habe, durchsehen konntet?\n\n\n\nTeacher: Es ist super, dass ihr euch die Materialien angesehen habt. Wir gehen die schwierigen Teile gemeinsam durch. Aber zuerst fassen wir kurz den ersten Abschnitt zusammen. Kann mir jemand die Hauptkonzepte von Thema 1 erklären?\n\n\n\nTeacher: Genau! Sehr gut. Und warum meint ihr, dass erneuerbare Energien so wichtig sind?\n\n\n\nTeacher: Das ist richtig! Erneuerbare Energien spielen eine große Rolle beim Schutz der Umwelt. Sie sind auch entscheidend für langfristige Lösungen unserer Energiebedürfnisse. Student01, du hast erwähnt, dass du eine Frage zu Thema 2 hast. Kannst du sie nochmal stellen?\n\n\n\nTeacher: Natürlich. Passive Solarheizung nutzt das Design von Gebäuden, um auf na

In [ ]:

if __name__ == "__main__":  
    if len(sys.argv) < 3:  
        print("Usage: python app.py <input_pdf_file> <output_txt_file>")  
        sys.exit(1)  
    
    input_pdf = sys.argv[1]  
    output_txt = sys.argv[2]  
    if len(sys.argv) == 4 and type(sys.argv[3]) == int:
        main(input_pdf, output_txt, sys.argv[3])
    else:
        main(input_pdf, output_txt)

Usage: python app.py <input_pdf_file> <output_txt_file>


SystemExit: 1

c:\ProgramData\anaconda3\envs\summarizor\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
